# 中文基于上下文的情感识别

In [1]:
import openai
import os

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
text = f"""
       <Context:Positive,
    Phrase: 当我第二次回来时，我的案件处理和解决的方式非常出色，我非常满意和开心。>
"""

In [4]:
prompt = f"""
    这是一项基于上下文的情绪分析任务。对于这个任务，有一个包含示例的教程，请按照教程的指导完成任务。以下是教程：
    <tutorial>
    阅读客户反馈中的短语(phrase)以及与该短语相关的上下文(context)，根据上下文（context）确定短语（phrase）的情感类别。

    短语（Phrase）和上下文（Context）在情感分析中的含义如下：

    - Phrase（短语）：客户评论中的短语。

    - context（上下文）：与客户评论相关的上下文，可分为四类：“positive context”, “neutral context”, “suggestion context”, and “negative context”。

    为了说明phrase和context的概念，请看以下示例（问题用作上下文context）和可能的答案（用作短语phrase）。

    示例1：你喜欢你的经历吗？

        回答1：服务很好

        答案2：服务速度

        回答3：没有，整体服务不好。

        答案4：酒店离街道很远。


        每个短语（phrase)的情感：

        回答1：“服务很好”是positive的

        答案2：“服务”是positive的，因为它推断客户喜欢服务。

        回答3：“没有，整体服务不好”是negative的，客户可能不同意正面反馈提示的问题，在这种情况下，他们想表达他们的意见，认为服务不好。

        答案4：“酒店离街道很远”是中性语境下的一种表述，在积极的背景下，这意味着这一声明是一种积极的情绪。（可能的原因：有些客人喜欢酒店远离街道，因为这里很安静）


    示例2：你对自己的经历有什么不喜欢的地方？

        回答1：服务不好

        答案2：服务速度

        回答3：整体服务不错，没有什么可抱怨的。

        答案4：酒店离街道很远

        每个短语(phrase)的情感：

        答案1：“服务不好”是negative的

        答案2：是negative的，因为可以推断客户不喜欢他们的服务速度。

        回答3：“整体服务不错，没有什么可抱怨的”是positive的。客户可能不同意负面反馈提示的问题。在这种情况下，他们想表达他们的意见，他们认为服务是好的。

        答案4：“酒店离街道很远”是一句话。在消极的背景下，这意味着这种说法是一种negative情绪。（有些客人可能不喜欢酒店远离街道，因为去某些地方不方便）

    这里有四种上下文（context），分别是：positive context, neutral context, negative context, and suggestion context，下面是关于上下文（context）的一些例子：

    1.Positive context
        - 这个经历中，你最喜欢的是什么吗？

    2.Neutral context
        - 你的总体经验是什么？
        - 你还有什么想问或告诉我们的吗？

    3.Negative context
        - 你有什么不喜欢你的经历？

    4.Suggestion context

        - 我们能做些什么来改善您的体验？

    以下是该短语在上下文条件下的情感标签：

    - No Clear Opinion：（无明确意见）事实陈述/描述或中性情绪

    - Positive（积极）该短语包含积极情绪

    - Strongly Positive（强烈正面）该短语包含强烈正面情绪

    - Negative（否定）该短语包含否定情绪

    - Strongly Negative（强烈否定）该短语包含强烈否定情绪

    - Suggestion（建议）该短语向公司提出采取行动的建议

    - Mixed Opinion （混合）该短语包含积极和消极情绪

    - Incorrect Language or Gibberish（语言错误或胡言乱语）该短语的语言不正确或完全胡言乱语

    - Unsure（不确定）：文本很清楚，但我不知道如何标注。如果你花了30多秒思考如何标注，请选择“不确定”。我们可能会为您提供答案和解释，但是，不要经常选择这个答案。\
        如果文本不清楚，因为它不是一个完整的句子，或者可能需要更多信息，请选择“无明确意见”
        

    以下是根据上下文（context)标注短语(phrase)的例子。

    1.No Clear Opinion

        在下面的示例中，所有phrase(短语)都被标记为No Clear Opinion

            1） 短语纯粹是对事实的观察、描述或陈述。
            示例：
            ＜Context: any,
            Phrase: 这是我们的周年纪念周末>
            Sentiment:No Clear Opinion
            
            2） 人们对产品或公司的看法是中立的。
            示例：
            ＜Context: Any,
            Phrase:产品还可以。>
            Sentiment：No Clear Opinion

            3） 如果这个(phrase)短语是描述性的，或者你可能需要更多的上下文（评论中的文本）来做出判断，请标记“No Clear Opinion”。这通常是由于分割不好、拼写错误太多以及文本本身并不意味着任何情感。
            示例：
            <Context: Any,
            Phrase: 我这么说就是为了说明这一点.>                
            Sentiment: No clear opinion

            4） 一个中性的问题，不意味着任何积极或消极的情绪。
            示例：
            <Context: Any,
            Phrase: 产品什么时候发布？>
            Sentiment: No clear opinion

            5） 在情绪清晰的短语（phrase)或在积极的上下文中，该短语（phrase)描述产品质量没有问题，或客户没有遇到任何问题，或他们的期望得到了满足。
            - 像软/硬、薄/厚或冷/热这样的形容词没有表现出固有的情感，应该被标记为中性或没有明确的意见，因为根据个人喜好，它可以是积极的，也可以是消极的。
            - 形容词如，好或真棒，本质上是积极的，应该被标记为积极情绪。
            - 像不好、可怕或糟糕这样本质上是负面的形容词应该被标记为负面情绪。
            示例：
            <Context: Neutral,
            Phrase: 布不透明!>      
            Sentiment: No clear opinion    
            
            <Context: Any,
            Phrase: 我喜欢这块不透明的布!>        
            Sentiment: Positive


            6） 这个短语只包含中性上下文中的名词。这个名词不包含任何积极/消极的情绪。他们只有在积极或消极的背景下才会变得积极或消极，但在中立的背景下却没有明确的意见。
            示例：
            <Context: Neutral,
            Phrase: 专业精神。>      
            Sentiment: No clear opinion


            7） 签字语言，如“谢谢”
            示例：
            <Context: Neutral,
            Phrase: 谢谢REDACT_NAME>        
            Sentiment: No clear opinion

        
    2. Positive:
        短语包含客户对客户体验的任何积极意见。在任何情况下，总是与良好的客户体验（“舒适的床”、“快速/快速的服务”、“友好/知识渊博/乐于助人的员工”、“便宜的产品/低成本”）相关的形容词都应该被标记为积极情绪。
        示例：
        <Context: Any,
        Phrase: 你准时交货.>
        Sentiment: Positive
        
        <Context: Neutral,
        Phrase: 床很硬>
        Sentiment: No clear opinion   

    3. Strongly Positive    
        这个短语通常含有表达积极情绪的强烈语气。客户可能会使用诸如“非常”、“极其”、“真的”之类的修饰语，或者任何诸如“棒极了”、“真棒”或“最佳”之类的强烈形容词。
        示例：
        <Context:Any,
        Phrase: 坐在Craps桌子旁的吉米太棒了，是第一次来的好老师！>
        Sentiment: Strongly positive
        
    4. Negative    
        短语包含：
        1） 客户对客户体验的任何负面意见（愤怒、沮丧、不满）或
        2） 客户遇到的任何问题

        总是与糟糕的客户体验有关的形容词（“不舒服的床”、“慢/差的服务”、“粗鲁的员工”、“昂贵的产品/高成本”、“不可接受的反应”）应该被标记为负面情绪。
            
        示例：
        <Context:Any,
        Phrase: 她似乎不理解或不想理解这个问题>
        Sentiment: Negative
            
    5. Strongly Negative    
        该短语通常含有强烈的负面情绪语气。客户可能会使用诸如“非常”、“极其”、“真的”、“最差”之类的修饰语，或者任何像“最差的”这样的强烈形容词。
        示例：
        <Context:Any,
        Phrase: 我很失望，因为我期待享受这顿自助餐已经很久了>
        Sentiment: Strongly negative

    6. Mixed Opinion
        这个短语既有积极的情绪，也有消极的情绪。
        示例：    
        <Context:Any,
        Phrase: 尽管餐厅的选择有限，但我们非常喜欢查理的食物，一天内在那里吃了两次.>
        Sentiment: Mixed opinion

        <Context:Any,
        Phrase: 虽然经纪人非常有礼貌和乐于助人，但我感到失望的是，您只准备向我们的客人退还一半的巨额预订费，这位客人之前曾与我们一起住过6次，由于2019冠状病毒病，他无法前往新西兰，不得不取消预订。>
        Sentiment: Mixed opinion

    7 Suggestion
        短语包含一些具体的行动或建议，以实现客户的愿望/意图:
            - 在任何情况下，一些常见的建议都有“这会很好/很棒”、“你需要”、“必须”、“应该”、“我希望”、“..会有帮助”、“…会帮助…”
            - 在消极或暗示的上下文中，以比较词（更好、更低、更少、更容易等）开头的短语意味着客户希望公司采取行动，使某些事情变得更好、更低、更少或更容易，因此这是一个建议。
            - 在suggestion上下文中，带有“能够做某事”或“有能力做某事”的短语意味着客户希望公司满足他们的需求，因此这是一个建议。
            - 短语包含“我希望”，但这并不是建议公司采取行动来满足客户的需求,这些短语是对问题或抱怨的描述。例如：
                --- 我想说的是服务很慢
                --- 我希望能给你打10分，但你们的服务很慢
    
        示例：
            <Context:Any,
            Phrase: 提供西班牙语移动应用程序>
            Sentiment: Suggestion       
        
    8. Incorrect language or gibberish
    如果文本的语言不正确，或者短语对你来说没有任何意义（胡言乱语），那么选择“incorrect language or gibberish”。如果只是句子的一部分是胡言乱语，但你仍然可以理解其中的含义，那么就选择你最好的情感判断。
    请注意，当我们从数据中删除个人敏感信息时，“%d”是替换敏感数字数据的符号。你应该把它们解释成一些数字，而不是incorrect language or gibberish。 
        示例：
        <Context:Any,
        Phrase: jjo>
        Sentiment: gibberish   

    9. Unsure
        文本对你来说很清楚，但你不知道如何注释它，也就是说，被划分为任何类别的概率几乎相等

    以上是教程的全部内容
    </tutorial>

    请进行基于上下文的情绪分析：
        - 阅读phrase(短语)和context(上下文)；由于数据隐私和安全原因，我们将用REDACT_company替换公司名称；电子邮件将替换为REDACT_EMAIL，任何可能包含个人数据的数字都被替换为“#”，因此您可能会在注释中看到重复的“###”。
        - 确定短语适用的情感类别。
        - 情绪类别应为： No Clear Opinion,positive, strongly Positive, negative, strongly Negative, suggestion, mixed Opinion, incorrect Language or Gibberish,and Unsure.
        - 请只输出情感的类别名称，不要输出更多的信息。

    上下文（context）和短语（phrase）: ```{text}```
"""


In [5]:
response = get_completion(prompt)
print(response)

positive
